In [1]:
import argparse
import numpy as np
import random
import argparse
import os
import time
import math
import sys
import json
import csv
import pandas as pd

import torch
from torch.autograd import Variable

from models import load_models_decode, generate
from utils import to_gpu, Corpus, batchify, train_ngram_lm, get_ppl

In [6]:
# load_path='output/out1102'
# load_path='output/out_twitter1103'
load_path='output/out1106_komoran'

In [7]:
model_args, idx2word, autoencoder, gan_gen, gan_disc, word2idx= load_models_decode(load_path)

Loading models fromoutput/out1106_komoran


In [436]:
vocab2=idx2word
vocab2

{0: '<pad>',
 1: '<sos>',
 2: '<eos>',
 3: '<oov>',
 4: '가',
 5: '가.',
 6: '가가',
 7: '가거나',
 8: '가게',
 9: '가게가도',
 10: '가게나',
 11: '가게되면',
 12: '가게되었으며',
 13: '가게마다',
 14: '가게에',
 15: '가게에서',
 16: '가게에서나',
 17: '가게주인이주어서.',
 18: '가격',
 19: '가격.',
 20: '가격과',
 21: '가격까지',
 22: '가격대',
 23: '가격대가',
 24: '가격대가있지만',
 25: '가격대도',
 26: '가격대로',
 27: '가격대비',
 28: '가격대에',
 29: '가격대와',
 30: '가격대중성.',
 31: '가격도',
 32: '가격도싸고',
 33: '가격면에서',
 34: '가격보다도',
 35: '가격부담없이',
 36: '가격에',
 37: '가격에다',
 38: '가격에서',
 39: '가격은',
 40: '가격을',
 41: '가격이',
 42: '가격이랑',
 43: '가격이싸고',
 44: '가격저렴.',
 45: '가격저렴하고',
 46: '가격적으로나',
 47: '가격적으로도',
 48: '가격적인',
 49: '가격차이',
 50: '가격치곤',
 51: '가고',
 52: '가공공법',
 53: '가기',
 54: '가까운',
 55: '가까운데서',
 56: '가까운맛.',
 57: '가까움',
 58: '가까움.',
 59: '가까이서',
 60: '가까이에서',
 61: '가깝게',
 62: '가깝다고',
 63: '가끔',
 64: '가끔가다',
 65: '가끔식',
 66: '가끔씩',
 67: '가나',
 68: '가는',
 69: '가는거같다.',
 70: '가는구간에도',
 71: '가는데',
 72: '가는듯한',
 73: '가능.',
 74: '가능하고',
 75: '가능하다',
 76: '가능하다.',
 77: '가능하며

In [437]:
word2idx

{'<pad>': 0,
 '<sos>': 1,
 '<eos>': 2,
 '<oov>': 3,
 '가': 4,
 '가.': 5,
 '가가': 6,
 '가거나': 7,
 '가게': 8,
 '가게가도': 9,
 '가게나': 10,
 '가게되면': 11,
 '가게되었으며': 12,
 '가게마다': 13,
 '가게에': 14,
 '가게에서': 15,
 '가게에서나': 16,
 '가게주인이주어서.': 17,
 '가격': 18,
 '가격.': 19,
 '가격과': 20,
 '가격까지': 21,
 '가격대': 22,
 '가격대가': 23,
 '가격대가있지만': 24,
 '가격대도': 25,
 '가격대로': 26,
 '가격대비': 27,
 '가격대에': 28,
 '가격대와': 29,
 '가격대중성.': 30,
 '가격도': 31,
 '가격도싸고': 32,
 '가격면에서': 33,
 '가격보다도': 34,
 '가격부담없이': 35,
 '가격에': 36,
 '가격에다': 37,
 '가격에서': 38,
 '가격은': 39,
 '가격을': 40,
 '가격이': 41,
 '가격이랑': 42,
 '가격이싸고': 43,
 '가격저렴.': 44,
 '가격저렴하고': 45,
 '가격적으로나': 46,
 '가격적으로도': 47,
 '가격적인': 48,
 '가격차이': 49,
 '가격치곤': 50,
 '가고': 51,
 '가공공법': 52,
 '가기': 53,
 '가까운': 54,
 '가까운데서': 55,
 '가까운맛.': 56,
 '가까움': 57,
 '가까움.': 58,
 '가까이서': 59,
 '가까이에서': 60,
 '가깝게': 61,
 '가깝다고': 62,
 '가끔': 63,
 '가끔가다': 64,
 '가끔식': 65,
 '가끔씩': 66,
 '가나': 67,
 '가는': 68,
 '가는거같다.': 69,
 '가는구간에도': 70,
 '가는데': 71,
 '가는듯한': 72,
 '가능.': 73,
 '가능하고': 74,
 '가능하다': 75,
 '가능하다.': 76,
 '가능하며': 7

In [400]:
word2idx['흔해서.']

10899

In [401]:
vocab2[10899]

'흔해서.'

In [8]:
model_args

{'N': 5,
 'arch_d': '300-300',
 'arch_g': '300-300',
 'batch_size': 5,
 'beta1': 0.9,
 'clip': 1,
 'cuda': True,
 'data_path': 'data_korean/komoran',
 'dropout': 0.0,
 'emsize': 300,
 'enc_grad_norm': True,
 'epochs': 49,
 'gan_clamp': 0.01,
 'gan_toenc': -0.01,
 'hidden_init': False,
 'kenlm_path': '../Data/kenlm',
 'log_interval': 200,
 'lowercase': False,
 'lr_ae': 1,
 'lr_gan_d': 1e-05,
 'lr_gan_g': 5e-05,
 'maxlen': 30,
 'min_epochs': 50,
 'nhidden': 300,
 'niters_ae': 1,
 'niters_gan_d': 5,
 'niters_gan_g': 1,
 'niters_gan_schedule': '2-4-6',
 'nlayers': 1,
 'no_earlystopping': False,
 'noise_anneal': 0.995,
 'noise_radius': 0.2,
 'ntokens': 2931,
 'outf': 'out1106_komoran',
 'patience': 5,
 'sample': False,
 'seed': 1111,
 'temp': 1,
 'vocab_size': 11000,
 'z_size': 100}

# ---------전처리

In [266]:
import re

In [254]:
file = 'codedata/rawinput.csv'
rawdata = pd.read_csv(file, names = None, encoding='CP949')
rawdoc = rawdata['응답값']

In [265]:
rawdoc1 = [" ".join(re.findall('([가-힣]+)', row))+"." for row in rawdoc]
rawdoc1
with open("train.txt", 'w') as f:
    for s in rawdoc1:
        f.write(str(s) + '\n')

# ------------------

In [438]:
path = 'codedata/clusterdata.csv'
maxlen=100

In [427]:
path = 'codedata/twitter_code_decode/clust_token_all_twitter.csv'
maxlen=100

In [440]:
data = pd.read_csv(path, names = None)
clusternum=data['ClusterNo']
doc = data['rawdoc']
groupby_cluster = data['rawdoc'].groupby([data['ClusterNo']])
cluster_doc=groupby_cluster.apply(lambda x: x.tolist()).to_dict()

In [380]:
cluster_doc

{0: ['시원해서.',
  '시원함.',
  '시원하다.',
  '시원함.',
  '시원해서.',
  '시원함.',
  '시원하다.',
  '시원함.',
  '시원해서.',
  '시원함.',
  '시원하다.',
  '시원해서.',
  '시원해서.',
  '시원함.',
  '시원함.',
  '시원하다.',
  '시원함.',
  '시원해서.',
  '시원함.',
  '시원함.',
  '시원함.',
  '시원하다.',
  '시원함.',
  '시원해서.',
  '시원해서.',
  '시원함.',
  '시원함.',
  '시원하다.',
  '시원하다.',
  '시원함.',
  '시원함.',
  '시원하다.',
  '시원함.',
  '시원함.',
  '시원하다.',
  '시원함.',
  '시원하다.',
  '시원함.',
  '시원하다.',
  '시원함.',
  '시원.',
  '시원해서.',
  '시원해서.',
  '시원하다.',
  '시원하다.',
  '시원하다.',
  '시원해서.',
  '시원.',
  '시원하다.',
  '시원해서.',
  '시원해서.',
  '시원함.',
  '시원하다.',
  '시원함.',
  '시원하다.',
  '시원하다.',
  '시원함.',
  '시원하다.',
  '시원하다.',
  '시원해서.',
  '시원함.',
  '시원함.',
  '시원하다.',
  '시원함.',
  '시원해서.',
  '시원하다.',
  '시원함.',
  '시원해요.',
  '시원함.',
  '시원함.',
  '시원함.',
  '시원함.',
  '시원해서.',
  '시원함.',
  '시원해서.',
  '시원해서.',
  '시원하다.',
  '시원함.',
  '시원함.'],
 1: ['맛이 깔끔하다.',
  '맛이 깔끔함.',
  '맛이 깔끔함.',
  '깔끔한 맛.',
  '맛이 깔끔하다.',
  '맛이 깔끔해서.',
  '맛이 깔끔함.',
  '맛이 깔끔함.',
  '맛이 깔끔하다.',
  '깔끔한 맛.',
  '깔끔한 맛.',
  '깔끔한 맛.',
  '깔끔한 

In [441]:
vocab = word2idx
ntokens = len(vocab)
def doc_to_line(cluster_doc, vocab):

    dropped = 0
    linecount = 0
    lines = []
    for line in cluster_doc:
        linecount += 1        
#         if self.lowercase:
#             words = line[:-1].lower().strip().split(" ")
#         else:
#             words = line[:-1].strip().split(" ")
        words = line[:-1].strip().split(" ")
        
        if len(words) > maxlen:
            dropped += 1
            continue
        words = ['<sos>'] + words
        words += ['<eos>']

        # vectorize

        unk_idx = vocab['<oov>']
        indices = [vocab[w] if w in vocab else unk_idx for w in words]

#         print(indices)
        lines.append(indices)
        
    return lines, vocab

In [431]:
def make_code(lines):
    eval_batch_size = 1
    test_data = batchify(lines, eval_batch_size, shuffle=False)
    
    
    
    code = []
    for i, batch in enumerate(test_data):
            source, target, lengths = batch
    #         source = to_gpu(True, Variable(source, volatile=True))
    #         target = to_gpu(True, Variable(target, volatile=True))
    #         mask = target.gt(0)
    #         masked_target = target.masked_select(mask)
    #         # examples x ntokens
    #         output_mask = mask.unsqueeze(1).expand(mask.size(0), ntokens)

            # output: batch x seq_len x ntokens
            output = autoencoder.encode(Variable(source), lengths, noise=False)
#             print(output)
            code.append(output)
    return code

In [365]:
feature_range=0.4


cluster_title=[]
for c in cluster_doc.keys() : 
    lines, vocab = doc_to_line(cluster_doc[c], vocab)
    code = make_code(lines)

    code_cat=torch.cat(code,0)
    
    #code_feature extract(절대값 0.1미만 0으로)
    code_cat_np = code_cat.data.numpy().squeeze()
    temp = np.copy(code_cat_np)
    temp[np.where(np.abs(temp)<feature_range)]=0
    code_feature=torch.FloatTensor(temp).view(-1,300)
    
    #code 평균
    code_mean=torch.mean(code_feature,0).view(1,-1) 

    max_indices = autoencoder.generate_code(code_mean, maxlen)
    max_indices = max_indices.data.cpu().numpy().squeeze()
    

    words = [vocab2[x] for x in max_indices.squeeze()]
#     print(words)
    # truncate sentences to first occurrence of <eos>
    truncated_sent = []
    for w in words:
        if w != '<eos>':
            truncated_sent.append(w)
        else:
            break
    sent = " ".join(truncated_sent)
    cluster_title.append(sent)

In [358]:
len(cluster_title)

536

In [366]:
resultdoc={}
for i in range(len(cluster_title)):
    resultdoc[i]=cluster_title[i]

In [352]:
resultdoc

{0: '하하. 청량감 향.',
 1: '맛 이 깔끔해서 풍부하다.',
 2: '부드러움 부드럽',
 3: '맛있어 요 부드러워',
 4: '목 넘김 이 부드럽 고 독특해서.',
 5: '톡 쏘는 맛 이 좋 다 그리고 탄산.',
 6: '편안하다. 가 있다.',
 7: '맛 이 너무 부드럽 아요.',
 8: '내 입맛 에 맛 잇어 서 익숙해서.',
 9: '맛 이 좋 아.',
 10: '산토리 맥주 가 대체 적 인 맛 에 끌림.',
 11: '목 넘김 이 좋 다 선호.',
 12: '맛 이 강하고 진하다 풍부함.',
 13: '그냥 좋 았 습니다.',
 14: '뒷맛 이 깔끔하고 풍부하다.',
 15: '시원하고 맛 도 가벼 움.',
 16: '맛 이 좋 고 색감',
 17: '맥주 가 맛 잇어 좋 아.',
 18: '목 넘김 이 좋 고 시원한 뒷맛.',
 19: '깔끔하고 맛 이 순하고 가벼워',
 20: '부드럽 고 진한 것 같고 고소함.',
 21: '거품 이 풍부해서 부드럽 고 맛 잇다.',
 22: '흑 맥주 를 좋아하고 기네스 가 더 훌륭함.',
 23: '맛 과 향 이 좋 다 독특해서.',
 24: '순하고 맛 이 너무 쓰',
 25: '상큼 한 맛 에 대한 고소함.',
 26: '쏘는 맛 이 인상 에 다',
 27: '맛 이 진하다 그리고 풍부함.',
 28: '가장 대중 적 이고 있으',
 29: '톡 쏘는 맛 이 좋 다 그리고 가격 이 높다.',
 30: '보리 의 맛 이 있어 서 시원한 맛 이라서.',
 31: '톡소 는 맛 이 좋 으며 선호.',
 32: '맛 이 산토리 와 다른 맛 이 일품.',
 33: '고급스러 움 병 색깔',
 34: '가격 도 저렴 하고 국산 맛 이 나 으니까.',
 35: '내 입맛 에 맛 는 그냥 그냥 먹음.',
 36: '버드와이저 맥주 보다 맛있어 보이 게 됨.',
 37: '자주 마시는 것 기 떄문 에 익숙하다.',
 38: '시원한 느낌 보다 시원한 느낌 이라서.',
 39: '익숙해서 부드럽 고 상쾌한

In [367]:
df = pd.DataFrame([resultdoc,cluster_doc]).T
df['clusterNo'] = df.index

In [361]:
df

,0,1,clusterNo
0,강예빈 의 청량감 참신 한 게 맛있어,"[시원해서., 시원함., 시원하다., 시원함., 시원해서., 시원함., 시원하다.,...",0
1,깔끔한 맛 이 강하지 고 세계,"[맛 이 깔끔하다., 맛 이 깔끔함., 맛 이 깔끔함., 깔끔한 맛., 맛 이 깔끔...",1
2,너무 목 넘김 또한 아주 좋아짐.,"[부드럽 다., 부드러움., 부드러움., 부드러움., 부드러움., 부드럽 다., 부...",2
3,맛있어 요 거품 에 적당한 것,"[맛있다., 맛있 음., 맛있어 서., 맛있어 서요., 맛있어 요., 맛있다., 맛...",3
4,목 넘김 이 부드럽 고 향 이 독특해서.,"[부드러운 목 넘김 이 좋 다 맛 도 부드러워 좋 다., 목 넘김 이 좋 고 부드럽...",4
5,톡 쏘는 맛 이 좋 다 그리고 톡 쏜다.,"[톡 쏘는 맛., 톡 쏘아 서., 맛 이 좋 음 톡 쏨., 톡 쏘는 맛 이 좋 다....",5
6,구하기 가 지금 깔 금하고 깨끗한 게 친숙함.,"[깔끔하다., 깔끔해 요., 깔끔함., 깔끔함., 깔끔해서., 깔끔해서., 깔끔하 ...",6
7,맛 이 너무 부드럽 다 가격 이.,"[부드러운 맛., 맛 이 부드러워 서., 부드러운 맛., 맛 이 부드럽 다., 맛 ...",7
8,맛 이 입맛 에 맞 음 고소함.,"[맛 이 입맛 에 맞아 서., 내 입맛 에 맞아 서., 내 입맛 에 맞 음., 맛 ...",8
9,맛 이 좋 다 풍부함.,"[맛 이 좋 다., 맛 이 좋 아서., 맛 이 좋 아서., 맛 이 좋 아서., 맛 ...",9


In [284]:
df

,0,1,clusterNo
0,시원 하 ㅁ 맛있 어요.,"[시원 하 아서., 시원 하 ㅁ., 시원 하 다., 시원 하 ㅁ., 시원 하 아서....",0
1,맛 이 깔끔 하 았 아.,"[맛 이 깔끔 하 다., 맛 이 깔끔 하 ㅁ., 맛 이 깔끔 하 ㅁ., 깔끔 하 ㄴ...",1
2,부드럽 ㅁ 아.,"[부드럽 다., 부드럽 ㅁ., 부드럽 ㅁ., 부드럽 ㅁ., 부드럽 ㅁ., 부드럽 다...",2
3,맛있 으니까.,"[맛있 다., 맛있 음., 맛있 어서., 맛있 어서요., 맛있 어요., 맛있 다.,...",3
4,목 넘기 ㅁ 이 부드럽 다 아.,"[부드럽 ㄴ 목 넘기 ㅁ 이 좋 다 맛 도 부드럽 어 좋 다., 목 넘기 ㅁ 이 좋...",4
5,톡 쏘 는 맛 순 하 아서 좋 아요.,"[톡 쏘 는 맛., 톡 쏘 아서., 맛 이 좋 음 톡 쏘 ㅁ., 톡 쏘 는 맛 이 ...",5
6,깔끔 하 아서 아.,"[깔끔 하 다., 깔끔 하 어요., 깔끔 하 ㅁ., 깔끔 하 ㅁ., 깔끔 하 아서....",6
7,부드럽 ㄴ 맛 이 담백.,"[부드럽 ㄴ 맛., 맛 이 부드럽 어서., 부드럽 ㄴ 맛., 맛 이 부드럽 다., ...",7
8,입맛 에 가장 맞 아서 아.,"[맛 이 입맛 에 맞 아서., 내 입맛 에 맞 아서., 내 입맛 에 맞 음., 맛 ...",8
9,맛 이 아서 좋 아.,"[맛 이 좋 다., 맛 이 좋 아서., 맛 이 좋 아서., 맛 이 좋 아서., 맛 ...",9


In [368]:
rows = []
_ = df.apply(lambda row: [rows.append([row['clusterNo'],row[0],  nn]) 
                         for nn in row[1]], axis=1)

In [369]:
df_new = pd.DataFrame(rows, columns=['clusterNo','title','raw_docs'])
df_new

,clusterNo,title,raw_docs
0,0,뒷 별 모양 이 맘 에 들어서 남자,시원해서.
1,0,뒷 별 모양 이 맘 에 들어서 남자,시원함.
2,0,뒷 별 모양 이 맘 에 들어서 남자,시원하다.
3,0,뒷 별 모양 이 맘 에 들어서 남자,시원함.
4,0,뒷 별 모양 이 맘 에 들어서 남자,시원해서.
5,0,뒷 별 모양 이 맘 에 들어서 남자,시원함.
6,0,뒷 별 모양 이 맘 에 들어서 남자,시원하다.
7,0,뒷 별 모양 이 맘 에 들어서 남자,시원함.
8,0,뒷 별 모양 이 맘 에 들어서 남자,시원해서.
9,0,뒷 별 모양 이 맘 에 들어서 남자,시원함.


In [370]:
df_new.to_csv('1114_twitter_generate_title_result_raw_0.4.csv',sep=',',encoding='CP949')

# ----------------------------code_vec

In [442]:
cluster_title=[]
code_vec=[]
l=0
for c in cluster_doc.keys() : 
    lines, vocab= doc_to_line(cluster_doc[c], vocab)
    eval_batch_size = 1
    
    test_data = batchify(lines, eval_batch_size, shuffle=False)
    
    ntokens = len(vocab)
    for i, batch in enumerate(test_data):
        
        source, target, lengths = batch
    #         source = to_gpu(True, Variable(source, volatile=True))
    #         target = to_gpu(True, Variable(target, volatile=True))
    #         mask = target.gt(0)
    #         masked_target = target.masked_select(mask)
    #         # examples x ntokens
    #         output_mask = mask.unsqueeze(1).expand(mask.size(0), ntokens)
        indices = Variable(source)
            # output: batch x seq_len x ntokens
        output = autoencoder.encode(indices, lengths, noise=False)
        
        code_vec.extend(output.data.numpy().tolist())
    l+=len(lines)

In [443]:
len(code_vec)

8524

In [444]:
df= pd.DataFrame(code_vec)
df1 = pd.concat([clusternum,doc,df], axis=1)
df1.to_csv('cluster_code_vec_raw_1115.csv', index=False)